<a href="https://colab.research.google.com/github/ArcherX0X/ECS171_Final_Project/blob/main/ECS171__Final_Project_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving features_30_sec.csv to features_30_sec (1).csv


In [ ]:
df = pd.read_csv('/content/features_3_sec.csv')
print(df.shape)
df = df.drop('filename', axis=1)
df = df.drop('length', axis=1)
y = df.pop('label').values
print(type(y))
df.head(10)

(9990, 60)
<class 'numpy.ndarray'>


,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,1.080790e+06,...,-2.853603,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767
1,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,6.722448e+05,...,4.074709,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081
2,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,7.907127e+05,...,4.806280,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122
3,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,9.216524e+05,...,-1.359111,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678
4,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,6.102111e+05,...,2.092937,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694
5,0.376670,0.089702,0.132618,0.003583,1994.915219,211700.619569,2152.767854,74263.873102,4371.985614,1.067105e+06,...,2.127198,31.448069,-3.448373,34.284130,-0.416165,40.791092,-3.649625,32.457901,3.025218,28.892687
6,0.379909,0.088827,0.130335,0.003166,1962.150096,177443.070045,2146.503479,98020.541422,4325.026668,1.172363e+06,...,4.893660,33.954071,-2.068194,25.623655,1.428141,47.957699,-3.267124,39.382240,3.276939,25.999132
7,0.331880,0.092119,0.140600,0.002546,1701.890924,35678.130616,1979.387612,36670.725886,3625.280386,3.174617e+05,...,3.122667,38.456211,-3.637886,24.530296,-0.105148,26.716150,-2.016985,23.150423,0.210787,42.512966
8,0.347877,0.094209,0.133130,0.002538,1746.473502,138073.931244,1887.619723,117069.920049,3586.934721,1.057633e+06,...,-0.285936,44.311455,-4.370029,29.873167,2.114592,33.843155,-2.264663,80.812393,3.758598,97.618835
9,0.358061,0.082957,0.115312,0.001846,1763.948942,61493.423121,1874.195710,51944.921435,3505.522649,4.451579e+05,...,6.691686,43.967834,-3.448304,48.671944,0.099792,41.839546,-7.677177,96.253654,0.791776,40.416420


In [ ]:
# 80:20
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42, shuffle=True)

# Fits the scaler on the training features
scaler = MinMaxScaler()
scaler.fit(X_train)

# Transforms the training and testing features
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Encodes labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Converts to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

Only additional Hidden Layers

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = nn.functional.softmax(x, dim=1)  # softmax
        return x

# Hyperparameters
input_size = X_train.shape[1]
hidden_size1 = 512  # First hidden layer
hidden_size2 = 256  # Second hidden layer
hidden_size3 = 128  # Third hidden layer
num_classes = len(label_encoder.classes_)
learning_rate = 0.002
num_epochs = 200
batch_size = 32

# Initializes model, loss function, and optimizer
model = Classifier(input_size, hidden_size1, hidden_size2, hidden_size3, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/200], Loss: 2.0006
Epoch [2/200], Loss: 2.0202
Epoch [3/200], Loss: 2.0861
Epoch [4/200], Loss: 2.1922
Epoch [5/200], Loss: 2.0956
Epoch [6/200], Loss: 1.7823
Epoch [7/200], Loss: 2.1376
Epoch [8/200], Loss: 1.9746
Epoch [9/200], Loss: 1.9827
Epoch [10/200], Loss: 1.9265
Epoch [11/200], Loss: 1.8463
Epoch [12/200], Loss: 1.8841
Epoch [13/200], Loss: 2.0289
Epoch [14/200], Loss: 1.7117
Epoch [15/200], Loss: 1.8214
Epoch [16/200], Loss: 1.6857
Epoch [17/200], Loss: 1.9284
Epoch [18/200], Loss: 1.9286
Epoch [19/200], Loss: 1.7889
Epoch [20/200], Loss: 1.5598
Epoch [21/200], Loss: 1.8465
Epoch [22/200], Loss: 1.7497
Epoch [23/200], Loss: 1.5399
Epoch [24/200], Loss: 1.7507
Epoch [25/200], Loss: 1.6862
Epoch [26/200], Loss: 1.8216
Epoch [27/200], Loss: 1.8156
Epoch [28/200], Loss: 1.6626
Epoch [29/200], Loss: 1.8855
Epoch [30/200], Loss: 1.5538
Epoch [31/200], Loss: 1.8748
Epoch [32/200], Loss: 1.8827
Epoch [33/200], Loss: 1.7127
Epoch [34/200], Loss: 1.6709
Epoch [35/200], Loss: 1

In [ ]:
# Evaluates the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    probabilities, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f'Accuracy: {accuracy:.4f}')

    y_test_decoded = label_encoder.inverse_transform(y_test_tensor)
    predicted_decoded = label_encoder.inverse_transform(predicted)

    print("Classification Report:")
    print(classification_report(y_test_decoded, predicted_decoded))

Accuracy: 0.7913
Classification Report:
              precision    recall  f1-score   support

       blues       0.91      0.66      0.77       208
   classical       0.87      0.98      0.92       203
     country       0.58      0.85      0.69       186
       disco       0.75      0.76      0.75       199
      hiphop       0.97      0.62      0.76       218
        jazz       0.86      0.80      0.83       192
       metal       0.89      0.88      0.89       204
         pop       0.76      0.93      0.83       180
      reggae       0.75      0.79      0.77       211
        rock       0.73      0.68      0.70       197

    accuracy                           0.79      1998
   macro avg       0.81      0.79      0.79      1998
weighted avg       0.81      0.79      0.79      1998



With additional Hidden Layers + DropOut

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes, dropout_rate=0.5):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_rate)
        self.fc4 = nn.Linear(hidden_size3, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)  # Apply dropout
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)  # Apply dropout
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)  # Apply dropout
        x = self.fc4(x)
        x = nn.functional.softmax(x, dim=1)  # softmax
        return x


In [ ]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size1 = 512  # First hidden layer
hidden_size2 = 256  # Second hidden layer
hidden_size3 = 128  # Third hidden layer
num_classes = len(label_encoder.classes_)
learning_rate = 0.002
num_epochs = 200
batch_size = 32

# Initializes model, loss function, and optimizer
model = Classifier(input_size, hidden_size1, hidden_size2, hidden_size3, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Evaluates the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    probabilities, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f'Accuracy: {accuracy:.4f}')

    y_test_decoded = label_encoder.inverse_transform(y_test_tensor)
    predicted_decoded = label_encoder.inverse_transform(predicted)

    # Generate classification report
    print("Classification Report:")
    print(classification_report(y_test_decoded, predicted_decoded))


Epoch [1/200], Loss: 2.1257
Epoch [2/200], Loss: 2.1336
Epoch [3/200], Loss: 2.0260
Epoch [4/200], Loss: 1.8987
Epoch [5/200], Loss: 2.0662
Epoch [6/200], Loss: 2.1309
Epoch [7/200], Loss: 2.1642
Epoch [8/200], Loss: 1.9369
Epoch [9/200], Loss: 1.8597
Epoch [10/200], Loss: 2.0145
Epoch [11/200], Loss: 2.0217
Epoch [12/200], Loss: 1.8480
Epoch [13/200], Loss: 1.9279
Epoch [14/200], Loss: 1.9764
Epoch [15/200], Loss: 1.9500
Epoch [16/200], Loss: 2.0690
Epoch [17/200], Loss: 1.9853
Epoch [18/200], Loss: 1.7095
Epoch [19/200], Loss: 2.0744
Epoch [20/200], Loss: 1.8187
Epoch [21/200], Loss: 1.9269
Epoch [22/200], Loss: 2.0394
Epoch [23/200], Loss: 1.8650
Epoch [24/200], Loss: 2.0541
Epoch [25/200], Loss: 1.9005
Epoch [26/200], Loss: 1.9457
Epoch [27/200], Loss: 2.0803
Epoch [28/200], Loss: 1.9516
Epoch [29/200], Loss: 2.1116
Epoch [30/200], Loss: 1.8754
Epoch [31/200], Loss: 1.9299
Epoch [32/200], Loss: 1.7957
Epoch [33/200], Loss: 1.9605
Epoch [34/200], Loss: 1.9565
Epoch [35/200], Loss: 1

Original Version

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size1, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, num_classes)


    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)

        x = nn.functional.softmax(x, dim=1)  # softmax
        return x

In [ ]:
# Hyperparameters
input_size = X_train.shape[1]
hidden_size1 = 512
num_classes = len(label_encoder.classes_)
learning_rate = 0.002
num_epochs = 200
batch_size = 32

# Initializes model, loss function, and optimizer
model = Classifier(input_size, hidden_size1, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Evaluates the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    probabilities, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print(f'Accuracy: {accuracy:.4f}')

    # Decode labels
    y_test_decoded = label_encoder.inverse_transform(y_test_tensor)
    predicted_decoded = label_encoder.inverse_transform(predicted)

    # Generate classification report
    print("Classification Report:")
    print(classification_report(y_test_decoded, predicted_decoded))


Epoch [1/200], Loss: 2.0383
Epoch [2/200], Loss: 1.9397
Epoch [3/200], Loss: 1.9136
Epoch [4/200], Loss: 1.8296
Epoch [5/200], Loss: 1.8861
Epoch [6/200], Loss: 1.7510
Epoch [7/200], Loss: 1.9606
Epoch [8/200], Loss: 1.8541
Epoch [9/200], Loss: 1.7889
Epoch [10/200], Loss: 1.8757
Epoch [11/200], Loss: 1.8331
Epoch [12/200], Loss: 1.7500
Epoch [13/200], Loss: 1.7985
Epoch [14/200], Loss: 1.9090
Epoch [15/200], Loss: 1.8284
Epoch [16/200], Loss: 1.7949
Epoch [17/200], Loss: 1.6542
Epoch [18/200], Loss: 1.7008
Epoch [19/200], Loss: 1.7745
Epoch [20/200], Loss: 1.7194
Epoch [21/200], Loss: 1.8940
Epoch [22/200], Loss: 1.7569
Epoch [23/200], Loss: 1.7152
Epoch [24/200], Loss: 1.5879
Epoch [25/200], Loss: 1.7896
Epoch [26/200], Loss: 1.7216
Epoch [27/200], Loss: 1.6365
Epoch [28/200], Loss: 1.6437
Epoch [29/200], Loss: 1.6816
Epoch [30/200], Loss: 1.6761
Epoch [31/200], Loss: 1.7647
Epoch [32/200], Loss: 1.5677
Epoch [33/200], Loss: 1.9188
Epoch [34/200], Loss: 1.5937
Epoch [35/200], Loss: 1